In [57]:
import os
import json
import requests
import base64
from datetime import date
import re
import time
from pprint import pprint
from openpyxl import load_workbook
import xlsxwriter
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyotp import TOTP
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver

In [58]:
brand = "Maxton Design"
print(f"Automated fulfillment started {brand}")

Automated fulfillment started Maxton Design


In [59]:
#Login Details

GORGIAS_URL = "https://mlperformance-1.gorgias.com"
GORGIAS_OD_USERNAME = 'amanina@mlperformance.co.uk'
GORGIAS_OD_API_KEY = 'a40765a261a5096b6c4f04d16b25bf655cb77edb6c397ee88b8267480caa54b8'

AUTHORIZATION = base64.b64encode(
    f"{GORGIAS_OD_USERNAME}:{GORGIAS_OD_API_KEY}".encode("utf-8")
).decode("utf-8")

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Basic {AUTHORIZATION}",
}
# print(HEADERS)

#shopify
SHOPIFY_ADMIN_URL = "https://admin.shopify.com/store/mlperformance/purchase_orders"
SHOPIFY_EMAIL = "jarvis@mlperformance.co.uk"
SHOPIFY_PASSWORD = "Te@mMLP2024!"
SHOPIFY_SECRET_KEY = "Y55XYVN4MULN372ISPYVK4B7WRG22R6N"

In [60]:
# Function to initialize Selenium browser for Shopify login
def initiate_browser(email, password, secret_key=None):
    browser = Driver(incognito=True)
    browser.get("https://admin.shopify.com/store/mlperformance/orders?inContextTimeframe=none")
    wait = WebDriverWait(browser, 30)
    
    # Enter email
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="account_email"]'))).send_keys(email)
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".captcha__submit"))).click()
    
    # Enter password
    time.sleep(2)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-password__input"))).send_keys(password)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".captcha__submit"))).click()
    
    # Handle 2FA if secret_key is provided
    if secret_key:
        from pyotp import TOTP
        auth_field = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input#account_tfa_code")))
        totp = TOTP(secret_key)
        token = totp.now()
        auth_field.send_keys(token)
        time.sleep(3)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-button--full-width"))).click()
    
    return browser

print("Browser initialization function defined.")

Browser initialization function defined.


In [61]:
#Supplier details

supplier_list = {"Maxton Design": {
        "supplier_id" : 143818911, # supplier id in po shopify
        "supplier_email" : ["sales@maxtondesign.co.uk"],
        "customer_ids":[175694695], # cutomer id in gorgias
        "subject_pattern": "MAXTON DESIGN UK - Order \\d+ dispatched"
    }
}

In [62]:
def generate_ticket_list(tag_id):
    subject_pattern = supplier_list["Maxton Design"]["subject_pattern"]
    customer_ids = supplier_list["Maxton Design"]["customer_ids"]
    ticket_id_list = []
    for customer_id in customer_ids:
        # supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}&order_by=created_datetime%3Adesc&page=1&per_page=30"
        supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}"
        response = requests.get(supplier_url, headers=HEADERS).json()
        pprint(response)
        
        ticket_id_list.extend(
            [
                data["id"]
                for data in response["data"]
                if (
                    re.search(subject_pattern, data["subject"])
                    # and any(tag_id in all_tag.values() for all_tag in data["tags"])
                    # and data["status"] == "open"
                )
            ]
        )
    return ticket_id_list

ticket_id_list = generate_ticket_list('711016')
ticket_id_list

{'data': [{'assignee_team': None,
           'assignee_user': {'bio': None,
                             'email': 'yonghan@mlperformance.co.uk',
                             'firstname': 'Yong',
                             'id': 375465955,
                             'lastname': 'Han',
                             'meta': {'profile_picture_url': None, 'sso': None},
                             'name': 'Yong Han'},
           'channel': 'email',
           'closed_datetime': None,
           'created_datetime': '2025-05-21T13:29:00+00:00',
           'custom_fields': {'31433': {'id': 31433,
                                       'value': 'Other::No Reply::Other'}},
           'customer': {'email': 'sales@maxtondesign.co.uk',
                        'firstname': 'MAXTON',
                        'id': 175694695,
                        'lastname': 'UK',
                        'meta': {'name_set_via': 'email'},
                        'name': 'MAXTON DESIGN UK'},
           'excerpt': 

[164936785,
 164925018,
 161958675,
 161958672,
 160853678,
 160853677,
 159842207,
 159840256,
 159840251,
 159667860]

In [64]:
# Fetch a single ticket from Gorgias
def get_single_ticket(ticket_id):
    url = f"{GORGIAS_URL}/api/tickets/{ticket_id}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to fetch ticket {ticket_id}: {response.status_code}")
        print(f"Error message: {response.text}")
        return None
    
    try:
        response_json = response.json()
        print(f"Raw API response for ticket {ticket_id}:")
        #pprint(response_json)  # Print
        
        # Try different response structures
        ticket = response_json.get("data") or response_json
        if not ticket:
            print(f"Ticket {ticket_id}: No ticket data found in response.")
            return None
        
        print(f"Successfully fetched ticket {ticket_id}.")
        return ticket
    except ValueError as e:
        print(f"Ticket {ticket_id}: Failed to parse JSON response: {str(e)}")
        print(f"Raw response: {response.text}")
        return None

ticket_data = {}

for single_ticket_id in ticket_id_list:
    # Fetch the ticket
    ticket = get_single_ticket(single_ticket_id)
    ticket_data[single_ticket_id] = ticket
    if ticket:
        print(f"Processed ticket data for {single_ticket_id}:")
        # pprint(ticket)  # Print processed ticket data
    else:
        print("No ticket data to process.")

Raw API response for ticket 164936785:
Successfully fetched ticket 164936785.
Processed ticket data for 164936785:
Raw API response for ticket 164925018:
Successfully fetched ticket 164925018.
Processed ticket data for 164925018:
Raw API response for ticket 161958675:
Successfully fetched ticket 161958675.
Processed ticket data for 161958675:
Raw API response for ticket 161958672:
Successfully fetched ticket 161958672.
Processed ticket data for 161958672:
Raw API response for ticket 160853678:
Successfully fetched ticket 160853678.
Processed ticket data for 160853678:
Raw API response for ticket 160853677:
Successfully fetched ticket 160853677.
Processed ticket data for 160853677:
Raw API response for ticket 159842207:
Successfully fetched ticket 159842207.
Processed ticket data for 159842207:
Raw API response for ticket 159840256:
Successfully fetched ticket 159840256.
Processed ticket data for 159840256:
Raw API response for ticket 159840251:
Successfully fetched ticket 159840251.
Pr

In [65]:
# Function to extract Reference Number 1 from ticket
def extract_reference_number(ticket, ticket_id):
    if not ticket:
        print(f"Ticket {ticket_id}: No data to process.")
        return None, ticket_id
    
    messages = ticket.get("messages", [])
    if not messages:
        print(f"Ticket {ticket_id}: No messages found.")
        return None, ticket_id
    
    body_text = messages[0].get("body_text", "")
    # Debug: Print body text excerpt
    body_excerpt = body_text[:200] + "..." if len(body_text) > 200 else body_text
    # print(f"Ticket {ticket_id} body excerpt:\n{body_excerpt}")
    
    match = re.search(r"Order ID:\s*(\d+)", body_text)
    match2 = re.search(r"Tracking number:\s*(\d+)", body_text)
    if match:
        reference_num = match.group(1)
        tracking_num = match2.group(1)
        # print(f"Ticket :{ticket_id}, Reference Number 1: {reference_num}, Tracking Number: {tracking_num}")
        # print(f"Full body text:\n{body_text}")
        return reference_num, ticket_id, tracking_num
    else:
        print(f"Ticket {ticket_id}: Reference Number 1 not found.")
        # print(f"Full body text:\n{body_text}")
        return None, ticket_id

# Extract reference number
ticket_info = []

for single_ticket_id, ticket in ticket_data.items():
    if ticket:
        ref_num, ticket_id, trac_num = extract_reference_number(ticket, single_ticket_id)
        ticket_link = f"{GORGIAS_URL}/app/ticket/{ticket_id}"
        if ref_num:
            ticket_info.append({"ticket_id": ticket_id, "reference_num": ref_num, "tracking_number": {trac_num}, "ticket_link": ticket_link})
            print(f"Stored: Ticket ID: {ticket_id}, Reference Number: {ref_num}, Tracking Number: {trac_num}, Link: {ticket_link}")
        else:
            print(f"Ticket ID: {ticket_id}, No reference number extracted, Link: {ticket_link}")
    else:
        print("Skipping extraction due to missing ticket data.")

Stored: Ticket ID: 164936785, Reference Number: 253807, Tracking Number: 1, Link: https://mlperformance-1.gorgias.com/app/ticket/164936785
Stored: Ticket ID: 164925018, Reference Number: 253830, Tracking Number: 7051188533, Link: https://mlperformance-1.gorgias.com/app/ticket/164925018
Stored: Ticket ID: 161958675, Reference Number: 251323, Tracking Number: 7049511519, Link: https://mlperformance-1.gorgias.com/app/ticket/161958675
Stored: Ticket ID: 161958672, Reference Number: 251322, Tracking Number: 7049511516, Link: https://mlperformance-1.gorgias.com/app/ticket/161958672
Stored: Ticket ID: 160853678, Reference Number: 249308, Tracking Number: 7048850908, Link: https://mlperformance-1.gorgias.com/app/ticket/160853678
Stored: Ticket ID: 160853677, Reference Number: 249305, Tracking Number: 7048850898, Link: https://mlperformance-1.gorgias.com/app/ticket/160853677
Stored: Ticket ID: 159842207, Reference Number: 249312, Tracking Number: 7048363147, Link: https://mlperformance-1.gorgia

In [87]:
# Initialize browser for Shopify
browser = initiate_browser(SHOPIFY_EMAIL, SHOPIFY_PASSWORD, SHOPIFY_SECRET_KEY)
if browser:
    print("Browser initialized and logged into Shopify.")
else:
    print("Browser initialization failed. Cannot proceed with PO search.")

Browser initialized and logged into Shopify.


In [88]:
# Function to search for PO in Shopify by reference number
def get_po_link(reference_num, browser):
    if not browser:
        print(f"Cannot search for PO with reference {reference_num}: Browser not initialized.")
        return "Browser not initialized"
    
    query_ref = f"{reference_num}"  
    search_url = f"{SHOPIFY_ADMIN_URL}?selectedView=all&query={query_ref}"
    
    browser.get(search_url)
    wait = WebDriverWait(browser, 10)
    
    try:
        po_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-primary-link="true"]'))
        ).get_attribute("href")
        print(f"Found PO for reference {query_ref}: {po_link}")
        return po_link
    except Exception as e:
        # print(f"Error finding PO for reference {query_ref}: {str(e)}")
        print(f"Error finding PO for reference {query_ref}")
        return "PO not found"
    
def get_po_tags(po_url, browser):
    wait = WebDriverWait(browser, 10)
    try:
        browser.get(po_url)
        tag_list = []
        tags = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Polaris-Tag__Text'))
        )
        for itag in tags:
            shop_tag = itag.get_attribute('innerText')
            tag_list.append(shop_tag)
        
        if 'Dropship' in tag_list:
            tag0 = 'Dropship'
        else:
            tag0 = 'Warehouse'
        print(f"Found tag: {tag0}")
        
        comment_list = []
        comments = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '._CommentText_1x55c_28'))
        )
        for icomment in comments:
            comment = icomment.get_attribute('innerText')
            if 'MLP' in comment or 'SCP' in comment:
                comment_list.append(comment)
        
        target_comment = ''.join(comment_list[-1])
        order_id = ' '.join(word for word in target_comment.split() if 'MLP' in word or 'SCP' in word).replace('#','').replace('<h1>','').replace('</h1>','')

        return tag0, order_id
    
    except Exception as e:
        # print(f"Error finding tag for reference {po_url}: {str(e)}")
        print(f"Error finding tag for reference {po_url}")
        return "Tag not found","Order ID not found"

def get_order_id_link(order_no, browser):
    browser.get(f'https://admin.shopify.com/store/mlperformance/orders?start=MQ%3D%3D&query={order_no}')
    wait = WebDriverWait(browser, 10)
    try:
        order_link = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("href")

        seacrh_od_id = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.Polaris-Link--removeUnderline'))
        ).get_attribute("innerText").strip()

        if order_no in seacrh_od_id:
            return order_link
        else:
            return 'Order link not found'
    except:
        return 'Order link not found'


In [92]:
# Search for POs
final_data = {}

if ticket_info and browser:
    for info in ticket_info:
        reference_num = info["reference_num"]
        ticket_id = info["ticket_id"]
        ticket_link = info["ticket_link"]
        tracking_num = info["tracking_number"]
        po_link = get_po_link(reference_num, browser)
        tag, order_id = get_po_tags(po_link, browser)
        order_url = get_order_id_link(order_id, browser)

        print(f"Ticket ID: {ticket_id}, Reference Number: {reference_num}, Ticket Link: {ticket_link}, Tracking Number: {trac_num}, PO Link: {po_link}, Tag: {tag}, Order ID: {order_id}, Order url: {order_url}")
        print('\n')
            
        final_data[ticket_id] = {
            "reference_number": reference_num,
            "tracking_number": str(tracking_num),
            "po_url": po_link,
            "tag": tag,
            "order_id": order_id,   # You can fill this later
            "order_url": str(order_url)
        }

        # for k, v in final_data.items():
        #     for key, val in v.items():
        #         if isinstance(val, set):
        #             print(f"Found set in ticket {k} at field '{key}': {val}")

    

    with open("po_data-muaz.json", "w") as f:
        json.dump(final_data, f, indent=4)
else:
    if not ticket_info:
        print("No ticket info to process for PO search.")
    if not browser:
        print("No browser initialized for PO search.")

Found PO for reference 253807: https://admin.shopify.com/store/mlperformance/purchase_orders/10166108543
Found tag: Warehouse
Ticket ID: 164936785, Reference Number: 253807, Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164936785, Tracking Number: 287617795546, PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10166108543, Tag: Warehouse, Order ID: MLPUS10585, Order url: https://admin.shopify.com/store/mlperformance/orders/11653278007679


Found PO for reference 253830: https://admin.shopify.com/store/mlperformance/purchase_orders/10137403775
Found tag: Dropship
Ticket ID: 164925018, Reference Number: 253830, Ticket Link: https://mlperformance-1.gorgias.com/app/ticket/164925018, Tracking Number: 287617795546, PO Link: https://admin.shopify.com/store/mlperformance/purchase_orders/10137403775, Tag: Dropship, Order ID: MLP143464, Order url: https://admin.shopify.com/store/mlperformance/orders/11636668989823


Found PO for reference 251323: https://admin.